## Before running this notebook, you must have already the numerically simulated waves associated to the representative cases of synthetic simulated TCs (obtained with MaxDiss algorithm in notebook 04)

inputs required: 
  * Synthetic simulation of historical TCs parameters (copulas obtained in notebook 04)
  * MaxDiss selection of synthetic simulated TCs (parameters obtained in notebook 04)
  * simulated waves for the above selected TCs (**outside TeslaKit**)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit 
from teslakit.database import Database
from teslakit.io.matlab import ReadMatfile
from teslakit.rbf import RBF_Reconstruction



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('GUAM')
           

In [8]:
# --------------------------------------
# load data and set parameters

# TCs copula simulated (dataset) and TCs MDA selection
xds_dataset = db.Load_TCs_r2_sim_params()
xds_subset = db.Load_TCs_r2_mda_params()

# r2 mda TCs solved simulations (not solved inside teslakit)
xds_TCs_sims = db.Load_TCs_r2_mda_Simulations()


<xarray.Dataset>
Dimensions:        (storm: 100000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables:
    pressure_mean  (storm) float64 ...
    pressure_min   (storm) float64 ...
    gamma          (storm) float64 ...
    delta          (storm) float64 ...
    velocity_mean  (storm) float64 ...


In [10]:
# --------------------------------------
# load custom RBFs training subset and dataset to interpolate

# TODO: our datasets (db.Load_TCs_r2_sim_params and db.Load_TCs_r2_mda_params) 
# do not match with currently stored solved simulations.

# we are using matching matlab files for KWAJALEIN TCs simulations
# (overriding subset so it matches solved target)
p_test = op.join(p_data, 'tests', 'RBF_GUAM')
p_subset = op.join(p_test, 'MDA_400_guam.mat')
#p_dataset = op.join(p_test, 'MULTIVARIATE_100000parameters.mat')
a = xds_dataset['pressure_mean'].values
b = xds_dataset['velocity_mean'].values
c = xds_dataset['gamma'].values
d = xds_dataset['delta'].values

dataset = np.stack((a,b,c,d), axis=1)

# load subset data
subset = ReadMatfile(p_subset)['Subset']  # pmean, vmean, gamma, delta

# load full dataset for interpolation
#dm = ReadMatfile(p_dataset)
#dataset= np.column_stack(
#    (dm['PMEAN1'], dm['VMEAN'], dm['GAMMA'], dm['DELTA'])
#)
print(np.shape(dataset))

(100000, 4)



## Simulated TCs -  Radial Basis Function

In [5]:
# --------------------------------------
# Extract waves data from TCs simulations (this is the RBFs training target)

print(xds_TCs_sims)
print()

# Normalize data
d_maxis = {}
d_minis = {}
xds_n = xds_TCs_sims.copy()
for k in ['hs', 'tp', 'ss', 'twl']:
    v = xds_n[k].values[:]
    mx = np.max(v)
    mn = np.min(v)
    xds_n[k] =(('storm',), (v-mn)/(mx-mn))
    
    # store maxs and mins for denormalization
    d_maxis[k] = mx
    d_minis[k] = mn

xds_n['dir'] = xds_n['dir'] * np.pi/180
    
print(xds_n)
print()


# Build RBF target numpy array
target = np.column_stack(
    (xds_n['hs'], xds_n['tp'], xds_n['ss'], xds_n['twl'], xds_n['dir'], xds_n['mu'])
)


<xarray.Dataset>
Dimensions:  (storm: 400)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    hs       (storm) float64 1.182 12.17 0.01416 0.3082 ... 7.627 0.1468 1.15
    tp       (storm) float64 9.283 13.89 1.499 5.714 ... 11.76 12.15 11.16 10.69
    dir      (storm) float64 235.2 42.54 239.0 13.96 ... 124.5 182.8 355.5 346.2
    ss       (storm) float64 0.002797 0.6178 0.007105 ... 0.239 0.01714 0.0224
    twl      (storm) float64 0.5377 3.186 0.01656 0.1713 ... 2.018 0.2438 0.6297
    mu       (storm) float64 0.5616 0.6201 0.7014 ... 0.8513 0.2757 0.3203

<xarray.Dataset>
Dimensions:  (storm: 400)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    hs       (storm) float64 0.08677 0.8938 0.001006 ... 0.5601 0.01074 0.08442
    tp       (storm) float64 0.4909 0.7813 0.0 0.2658 ... 0.672 0.6094 0.5793
    dir      (storm) float64 4.104 0.7424 4.172 0.2437 ... 3

In [6]:
# --------------------------------------
# RBF Interpolation

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]        # scalar (pmean,  vmean)
ix_directional_subset = [2,3]   # directional (delta, gamma)

# target - scalar / directional indexes
ix_scalar_target = [0,1,2,3,5]  # scalar (Hs, Tp, SS, TWL, MU)
ix_directional_target = [4]     # directional (Dir)

output = RBF_Reconstruction(
    subset, ix_scalar_subset, ix_directional_subset,
    target, ix_scalar_target, ix_directional_target,
    dataset)


ix_scalar: 0,  optimization: 23.29 | interpolation: 5.76
ix_scalar: 1,  optimization: 24.06 | interpolation: 6.04
ix_scalar: 2,  optimization: 22.00 | interpolation: 6.24
ix_scalar: 3,  optimization: 24.15 | interpolation: 6.50
ix_scalar: 5,  optimization: 24.92 | interpolation: 7.07
ix_directional: 4,  optimization: 46.88 | interpolation: 12.50


In [7]:
# --------------------------------------
# output de-normalization

hs_n = output[:,0]
tp_n = output[:,1]
ss_n = output[:,2]
twl_n = output[:,3]
dir_n = output[:,4]
mu = output[:,5]

# store data as xarray.Dataset
xds_out = xr.Dataset(
    {
        'hs':(('storm',), hs_n * (d_maxis['hs']-d_minis['hs']) + d_minis['hs'] ),
        'tp':(('storm',), tp_n * (d_maxis['tp']-d_minis['tp']) + d_minis['tp'] ),
        'ss':(('storm',), ss_n * (d_maxis['ss']-d_minis['ss']) + d_minis['ss'] ),
        'twl':(('storm',), twl_n * (d_maxis['twl']-d_minis['twl']) + d_minis['twl'] ),
        'dir':(('storm',), dir_n * 180 / np.pi),
        'mu':(('storm',), mu),
    },
    coords = {'storm': np.arange(output.shape[0])}
)
print(xds_out)

db.Save_TCs_sim_r2_rbf_output(xds_out)


<xarray.Dataset>
Dimensions:  (storm: 100000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 ... 99994 99995 99996 99997 99998 99999
Data variables:
    hs       (storm) float64 1.453 1.964 2.263 1.95 ... 1.417 2.101 2.11 1.768
    tp       (storm) float64 8.939 9.217 10.11 9.632 ... 8.366 9.926 8.529 9.06
    ss       (storm) float64 0.02754 0.04406 0.05683 ... 0.0532 0.04505 0.03462
    twl      (storm) float64 0.5727 0.6927 0.8229 0.7311 ... 0.7947 0.693 0.6438
    dir      (storm) float64 165.4 165.6 153.6 150.6 ... 163.7 147.3 131.0 149.2
    mu       (storm) float64 0.84 0.6972 0.7319 0.7137 ... 0.6995 0.6603 0.7349
